In [1]:
from pyspark.sql import SparkSession

In [2]:
%fs ls dbfs:/FileStore/tables

path,name,size
dbfs:/FileStore/tables/README.txt,README.txt,8342
dbfs:/FileStore/tables/iris_data-04fff.csv,iris_data-04fff.csv,4642
dbfs:/FileStore/tables/links.csv,links.csv,197979
dbfs:/FileStore/tables/model/,model/,0
dbfs:/FileStore/tables/movies.csv,movies.csv,494431
dbfs:/FileStore/tables/ratings.csv,ratings.csv,2483723
dbfs:/FileStore/tables/tags.csv,tags.csv,118660
dbfs:/FileStore/tables/test.data,test.data,142
dbfs:/FileStore/tables/train.csv,train.csv,61194


In [3]:
data = spark.read.csv('dbfs:/FileStore/tables/ratings.csv', inferSchema=True, header=True)

In [4]:
data.show()
data.describe()


+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
 1| 70| 3.0|964982400|
 1| 101| 5.0|964980868|
 1| 110| 4.0|964982176|
 1| 151| 5.0|964984041|
 1| 157| 5.0|964984100|
 1| 163| 5.0|964983650|
 1| 216| 5.0|964981208|
 1| 223| 3.0|964980985|
 1| 231| 5.0|964981179|
 1| 235| 4.0|964980908|
 1| 260| 5.0|964981680|
 1| 296| 3.0|964982967|
 1| 316| 3.0|964982310|
 1| 333| 5.0|964981179|
 1| 349| 4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows

 Out[ 6 ]: DataFrame[summary: string, userId: string, movieId: string, rating: string, timestamp: string]

In [5]:
# train test split
train_data, test_data = data.randomSplit([0.8,0.2])

In [6]:
# instantiate model
from pyspark.ml.recommendation import ALS
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating',coldStartStrategy='drop')


In [7]:
# train
als_model = als.fit(train_data)


In [8]:
# test
predictions = als_model.transform(test_data)
predictions.show()

+------+-------+------+----------+----------+
userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
 385| 471| 4.0| 850766697| 3.0721703|
 91| 471| 1.0|1112713817| 4.0130324|
 599| 471| 2.5|1498518822| 3.537553|
 474| 471| 3.0| 974668858| 4.137152|
 500| 471| 1.0|1005528017| 1.7068686|
 411| 471| 4.0| 835532928| 2.8676724|
 357| 471| 3.5|1348627082| 4.2704654|
 104| 471| 4.5|1238111129| 4.0144353|
 492| 833| 4.0| 863976674| 3.0728278|
 599| 1088| 2.5|1498515232| 2.172123|
 111| 1088| 3.0|1516153967| 4.338223|
 177| 1088| 3.5|1435534616| 3.521076|
 20| 1088| 4.5|1054147512| 4.068862|
 554| 1088| 5.0| 944900489| 3.6426415|
 381| 1088| 3.5|1168664508| 3.4716637|
 594| 1088| 4.5|1109035643| 5.2141457|
 555| 1088| 4.0| 978822670| 3.8722618|
 84| 1088| 3.0| 860398568| 3.802003|
 51| 1088| 4.0|1230929736| 3.0754051|
 509| 1088| 3.0|1435992808| 3.4773562|
+------+-------+------+----------+----------+
only showing top 20 rows

In [9]:
# evaluate
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')
rmse = evaluator.evaluate(predictions)
rmse

Out[ 13 ]: 1.0777512948534018

In [10]:
# make predictions for single user
single_user = test_data.filter(test_data['userId']==1).select(['movieId','userId'])
single_user.show()
recommendations = als_model.transform(single_user)
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+
movieId|userId|
+-------+------+
 47| 1|
 163| 1|
 216| 1|
 441| 1|
 500| 1|
 543| 1|
 590| 1|
 608| 1|
 661| 1|
 919| 1|
 923| 1|
 1042| 1|
 1060| 1|
 1073| 1|
 1089| 1|
 1214| 1|
 1219| 1|
 1256| 1|
 1258| 1|
 1348| 1|
+-------+------+
only showing top 20 rows

+-------+------+----------+
movieId|userId|prediction|
+-------+------+----------+
 3441| 1| 5.512255|
 441| 1| 5.4064555|
 1060| 1| 5.180235|
 1089| 1| 5.1449475|
 608| 1| 5.142498|
 923| 1| 5.099044|
 4006| 1| 4.9879127|
 3062| 1| 4.930221|
 1408| 1| 4.9165382|
 47| 1| 4.9141936|
 2406| 1| 4.871222|
 1258| 1| 4.859664|
 1927| 1| 4.8524404|
 1954| 1| 4.786693|
 1219| 1| 4.746825|
 3448| 1| 4.7174773|
 1214| 1| 4.696525|
 2414| 1| 4.678077|
 2115| 1| 4.629385|
 2761| 1| 4.6205077|
+-------+------+----------+
only showing top 20 rows